# 4b Commute Analysis

This module calculates commute-level metrics (peak 5-minute times; average, 50-, 80-, 90-, and 95-percentile travel times; vmt; and duration and cost of congestion). The resulting Excel files contain tables that are read into the SummaryStats Excel sheet and form the basis of the GP (and, for NWR only, HOV) appendix tables. The module also produces the congestion and severe congestion plots for each commute as well as travel time and speed plots for data exploration.

Before running this module, several steps need to be completed. First, *3a_Corridor_Analysis* needs to be completed for the corridors in the region(s) that you would like to process. The *[corridor] [region].dat* files output by that module will be used to calculate vmt in this module. Although it is not necessary to complete *4a_Commute_Setup*, completing that module will help eliminate potential errors/discrepancies in the commute files and prevent the need to re-run this analysis if you find data discrepancies in the results. Even if the *4a_Commute_Setup* step is not completed, the *[region]_Commutes.xlsx* file (located by default at *./[ccr]/0_Inputs/[region]*) for the region must be prepared and the commute files from TRAC need to be placed in an appropriate folder (the default path is *./[ccr]/1_Data/[region]/2_Commute Data/[year]/* but a different location may be used if specified in the *[region]_Commutes.xlsx* file).

For small regions (all but NWR) the computation does not take very long, so if you want to update any of your input files you can rebuild the entire Region object without taking too much time. However, for large regions (or if every second counts, even on a small region), then you can update specific Commute objects within the Region object. The commutes to **exclude** in an update may be chosen in the *[region]_Commutes.xlsx* file.

## Inputs

The inputs for this module are:

Base Year, Current Year : the base and current analysis years for the current analysis. (e.g. 2015, 2017)

CCR : the current CCR and name of the main folder for the current CCR (e.g. 'CCR 18')

Analyst : analyst's name

Suffix (in and out) : the *in* Suffix is the suffix for previously processed and written Region binary objects. The *out* Suffix is a tag to add to the filenames of the Excel and binary outputs of this module. For example, if Suffix = '_8-20-2018' then the Excel outputfile for NWR base-year would be named *NWR_commutes_base_8-20-2018.xlsx*.

Plot Types : the types of plot to output. Acceptable Plot Types are:   
hov_cong : plots same year congestion - HOV vs GP vs REV  
gp_cong : plots comparison of base and current year congestion  
gp_sc : plots comparison of base and current year severe congestion  
gp_tt : plots comparison of base and current year travel time  
gp_spd : plots comparison of base and current year speed  
**If the Plot Types input area is left blank, all types will be plotted.**

## Main Code Block

#### Import Dependencies

In [1]:
#---------------------------- import dependencies -----------------------------
import os
import pickle
import pymas
from pymas.corridor_tools import *
from pymas.pymas_classes import Region

# disable warnings
import warnings
warnings.filterwarnings('ignore')

#### Create Region object(s)

This cell creates a Region object for each region (selected in Batch) and creates Commute objects for the commutes within the region. 

In [4]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

# get inputs from 0_Interface.xlsx 
inputs = get_inputs('4b', interface, sheet)
base_year = inputs['base_year']
curr_year = inputs['curr_year']
ccr = inputs['ccr']
analyst = inputs['analyst']

regions = get_batchlist('commute')

# instantiate empty dictionary to store Region objects
objs = {}

for reg in regions:
    print('\nBuilding %s Region object'%reg)
    com_xl = '%s_Commutes.xlsx'%(reg) #commute inputs
    paths = define_paths(ccr, reg, base_year, curr_year)
    objs[reg] = Region(com_xl, paths, analyst)

print('\n\nDone building objs dictionary')


Building SWR Region object
Building I-5 (I-5 bridge [Oregon] to I-205 interchange) Commute object
Building I-5 (I-205 interchange to I-5 bridge [Oregon]) Commute object
Building I-5 (I-5 bridge [Oregon] to SR 500 interchange) Commute object
Building I-5 (SR 500 interchange to I-5 bridge [Oregon]) Commute object
Building I-205 (Glenn Jackson Bridge [Oregon] to I-5 interchange) Commute object
Building I-205 (I-5 interchange to Glenn Jackson Bridge [Oregon]) Commute object
Building I-205 (Glenn Jackson Bridge [Oregon] to SR 500 interchange) Commute object
Building I-205 (SR 500 interchange to Glenn Jackson Bridge [Oregon]) Commute object
Building SR 14 (Camas to I-5 interchange) Commute object
Building SR 14 (I-5 interchange to Camas) Commute object
Building SR 14 (Camas to I-205 interchange) Commute object
Building SR 14 (I-205 interchange to Camas) Commute object


Done building objs dictionary


#### Write objects to Excel and .dat

This cell writes all Region objects in the objs dictionary to Excel files and binary files.

In [5]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

# get inputs from 0_Interface.xlsx 
inputs = get_inputs('4b', interface, sheet)
base_year = inputs['base_year']
curr_year = inputs['curr_year']
ccr = inputs['ccr']
analyst = inputs['analyst']

suffix = inputs['suffix_out']

regions = get_batchlist('commute')

for reg, obj in objs.iteritems():
    if obj.name in regions:
        print('\nWriting %s_commutes%s.dat'%(reg, suffix))
        obj.export_excel(suffix=suffix)
        obj.export_dat(suffix=suffix)

print('\n\nDone writing files')


Writing SWR_commutes.dat


Done writing files


#### Build objs dictionary from Region object files

This cell builds the objs dictionary (either from scratch, or appends the existing objs dictionary if it exists) from binary files of Region objects. This is used if you want to re-load and update or otherwise continue the analysis of an already built Region object.

In [2]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

# get inputs from 0_Interface.xlsx 
inputs = get_inputs('4b', interface, sheet)
base_year = inputs['base_year']
curr_year = inputs['curr_year']
ccr = inputs['ccr']
analyst = inputs['analyst']

suffix = inputs['suffix_in']

regions = get_batchlist('commute')

# if necessary, instantiate empty dictionary to store Region objects
try:
    objs
except:
    objs = {}

for reg in regions:
    print('\nLoading %s%s Region object'%(reg, suffix))
    paths = define_paths(ccr, reg, base_year, curr_year)
    
    try:
        objs[reg] = read_object('%s_commutes%s'%(reg, suffix), 'Region', paths)
    except:
        print('\nDid not find %s%s object'%(reg, suffix))        
        
print('\n\nDone building objs dictionary')


Loading SWR Region object


Done building objs dictionary


#### Plot commute data

This cell plots the GP and HOV (if applicable) congestion plots as well as travel time and speed plots if desired. The travel time and speed plots are not published, but are useful in visualizing and understanding the data.

In [ ]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

# get inputs from 0_Interface.xlsx 
inputs = get_inputs('4b', interface, sheet)

plot_types = inputs['plot_types']

for name, obj in objs.iteritems():
    print('\nPlotting %s'%name)
    obj.read_com_xl()
    obj.plot(plot_types)

print('\n\nDone plotting')

#### Update Region Object

This cell will update Region objects in the objs dictionary. Updating means re-loading and analyzing all commutes indicated (by not being excluded with 'x') in the relevant *[region]_Commutes.xlsx* file.

For example, if I have already built a NWR Region object and later need to change one or more of the input files for the "I-5 Seattle to SeaTac" commute, I can use this cell to rebuild only this Commute object instead of rebuilding the entire region. To do this, I would use the "Build objs dictionary from Region object files" cell to read the NWR Region binary file into the objs dictionary, mark all but the "I-5 Seattle to SeaTac" commutes with 'x' in the *NWR_Commutes.xlsx* file (and save the file), then run this cell. This will rebuid only the desired Commute object instead of the entire Region object. The new Region object can then be plotted and exported again to Excel and .dat with the updated results.

In [ ]:
interface = '0_Interface.xlsx'
sheet = 'Inputs'

#------------------------------------------------------------------------------
# You shouldn't need to edit anything below this line
#------------------------------------------------------------------------------

suffix = inputs['suffix_out']

regions = get_batchlist('commute')

for reg, obj in objs.iteritems():
    if obj.name in regions:
        print('\nUpdating %s'%reg)
        obj.update()


print('Done updating Region(s)')